In [12]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('C://Users/kamel/Downloads/Nasa_Space_Apps_2023/New Folder/Mag.csv')
df.head()

,Unnamed: 0,dsc_time,dsc_bx_gse,dsc_by_gse,dsc_bz_gse,dsc_bx_gsm,dsc_by_gsm,dsc_bz_gsm,wind_bx_gsm,wind_by_gsm,wind_bz_gsm,wind_bx_gse,wind_by_gse,wind_bz_gse,Kp_index
0,0,1.638317e+12,-4.14748,2.57484,5.80215,-4.14748,2.48670,5.84047,0.088021,4.011500,4.443556,0.087816,4.110653,4.351998,4.000000
1,1,1.638317e+12,-4.38719,2.43669,5.85642,-4.38719,2.34778,5.89262,0.743966,4.511123,4.838581,0.743736,4.619002,4.735743,3.999935
2,2,1.638317e+12,-4.16530,2.82073,5.58057,-4.16530,2.73598,5.62260,1.551022,3.971076,4.476637,1.550803,4.070888,4.386144,3.999870
3,3,1.638317e+12,-4.40910,3.04584,5.99368,-4.40910,2.95486,6.03905,0.571862,3.344407,5.211400,0.571598,3.460871,5.134824,3.999804
4,4,1.638317e+12,-4.11849,3.35687,5.77462,-4.11849,3.26920,5.82470,0.985459,3.322290,5.867872,0.985153,3.453475,5.791687,3.999739


In [3]:
X = df.iloc[:, 8:-1]
X.head()

,wind_bx_gsm,wind_by_gsm,wind_bz_gsm,wind_bx_gse,wind_by_gse,wind_bz_gse
0,0.088021,4.011500,4.443556,0.087816,4.110653,4.351998
1,0.743966,4.511123,4.838581,0.743736,4.619002,4.735743
2,1.551022,3.971076,4.476637,1.550803,4.070888,4.386144
3,0.571862,3.344407,5.211400,0.571598,3.460871,5.134824
4,0.985459,3.322290,5.867872,0.985153,3.453475,5.791687


In [5]:
y = df['Kp_index'][:900000]
y.head()

0    4.000000
1    3.999935
2    3.999870
3    3.999804
4    3.999739
Name: Kp_index, dtype: float64

In [6]:
def train_test_split(X, y, testPercentage=0.2):
    num_rows = X.shape[0]
    num_test = int(num_rows * testPercentage)
    X_train, X_test = X[:-num_test], X[-num_test:]
    y_train, y_test = y[:-num_test], y[-num_test:]
    return X_train, X_test, y_train, y_test

In [7]:
X_train, X_test, y_train, y_test = train_test_split(np.array(X), np.array(y))

In [9]:
from transformers import TFBertModel
from tensorflow.keras import layers, Model

# Define the transformer decoder function
def transformer_decoder(input_layer, enc_output, d_model, nhead):
    attn_output1 = layers.MultiHeadAttention(num_heads=nhead, key_dim=d_model)(input_layer, input_layer)
    attn_output1 = layers.LayerNormalization(epsilon=1e-6)(attn_output1 + input_layer)
    attn_output2 = layers.MultiHeadAttention(num_heads=nhead, key_dim=d_model)(attn_output1, enc_output)
    attn_output2 = layers.LayerNormalization(epsilon=1e-6)(attn_output2 + attn_output1)
    ffn_output = layers.Dense(d_model)(attn_output2)
    return layers.LayerNormalization(epsilon=1e-6)(ffn_output + attn_output2)

# Load pre-trained BERT model
bert_encoder = TFBertModel.from_pretrained('bert-base-uncased')

# Make BERT layers non-trainable
for layer in bert_encoder.layers:
    layer.trainable = False

input1 = layers.Input(shape=(6, ), dtype='int32')

# BERT layer
bert_output = bert_encoder(input1)[0]

# Decoder part
dec_output = transformer_decoder(bert_output, bert_output, d_model=768, nhead=4)

# Fully connected layers
x = layers.Flatten()(dec_output)
x = layers.Dense(64)(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(32)(x)

# Output layer for regression
output = layers.Dense(1, activation='linear')(x)

# Compile the model
model = Model(inputs=input1, outputs=output)
model.compile(optimizer='adam', loss='mse')

# Print model summary
model.summary()

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 6)]          0           []                               
                                                                                                  
 tf_bert_model (TFBertModel)    TFBaseModelOutputWi  109482240   ['input_1[0][0]']                
                                thPoolingAndCrossAt                                               
                                tentions(last_hidde                                               
                                n_state=(None, 6, 7                                               
                                68),                                                              
                                 pooler_output=(Non                                           

In [ ]:
# Model Training
model_history = model.fit(X_train, y_train,
                          validation_data=(X_test, y_test),
                          batch_size=32,
                          epochs=1,
                          verbose=1)

In [11]:
model.save('kp_prediction.h5')